In [1]:
from open3d import utility
from open3d import geometry
from open3d import io
from open3d import visualization
import numpy as np
import math
import glob
import random
from ipywidgets import FloatProgress
from IPython.display import display

In [51]:
class Box:
    
    def __init__(self, points, grid_idx):
        self.points = points
        self.adjacent_boxes = []
        self.grid_idx = grid_idx
        self.mesh = None
    
    def triangulate(self, parameters):
        mesh = Mesh()
        mesh.vertices = self.points
        mesh.triangulate(parameters)
        self.mesh = mesh
        
    def convex_hull(self):
        points = [point.position() for point in self.points]
        if len(self.points) <= 3:
            return None
        pcd = geometry.PointCloud()
        pcd.points = utility.Vector3dVector(np.array(points))
        return geometry.compute_point_cloud_convex_hull(pcd)
    

class Grid:
    
    def __init__(self, points, density):
        self.density = density
        self.points = points
        self.boxes = []
        self.nonempty_boxes = []
        self.box_dimensions = None
        self.grid_dimensions = None
        self.total_boxes = None
        self.get_box_dimensions()
        self.create_grid()
    
    def get_box_dimensions(self):
        pcd = geometry.PointCloud()
        pcd.points = utility.Vector3dVector(self.points)
        tree = geometry.KDTreeFlann(pcd)
        box_boundaries = []
        for point in self.points:
            [_, neighbors_idx, _] = tree.search_knn_vector_3d(point, self.density)
            neighbor_points = []
            for idx in neighbors_idx:
                neighbor_points.append(self.points[idx])
            neighbor_points = np.asarray(neighbor_points)
            neighbor_boundary = neighbor_points.max(axis=0) - neighbor_points.min(axis=0)
            box_boundaries.append(neighbor_boundary)
        box_dimensions = np.mean(np.asarray(box_boundaries), axis=0)
        bounding_box = pcd.get_max_bound() - pcd.get_min_bound()
        grid_dimensions = (bounding_box/box_dimensions).astype(np.int64)
        box_dimensions = bounding_box/grid_dimensions
        total_boxes = grid_dimensions[0] * grid_dimensions[1] * grid_dimensions[2]
        self.box_dimensions = box_dimensions
        self.grid_dimensions = grid_dimensions
        self.total_boxes = total_boxes
        
    def get_nonempty_boxes(self):
        nonempty_boxes = []
        for box in self.boxes:
            if len(box.points) > 2:
                nonempty_boxes.append(box)
        self.nonempty_boxes = nonempty_boxes
    
    def get_adjacent_boxes(self):
        for current_box in self.nonempty_boxes:
            adjacent_boxes = []
            idx_y, idx_z, idx_x = current_box.grid_idx
            for i in range(-1,2):
                for j in range(-1,2):
                    for k in range(-1,2):
                        if not (i == 0 and j == 0 and k == 0):
                            for box in self.nonempty_boxes:
                                if box.grid_idx == (idx_y+i, idx_z+j, idx_x+k):
                                    adjacent_boxes.append(box)
            current_box.adjacent_boxes = adjacent_boxes
        
    def create_grid(self):
        print('|Starting Grid|\n')
        print(f'Number of Neighbors: {self.density}')
        print(f'Number of Boxes: {self.total_boxes}')
        print(f'Grid Dimensions: {self.grid_dimensions}')
        print(f'Box Dimensions: {self.box_dimensions}')
        
        points = self.points
        min_x, min_y, min_z = points.min(axis=0)
        current_x, current_y, current_z = min_x, min_y, min_z
        grid_x, grid_y, grid_z = self.grid_dimensions
        box_x, box_y, box_z = self.box_dimensions
        
        boxes = []

        f = FloatProgress(min=0, max=self.total_boxes)
        display(f)

        for idx_y in range(grid_y):
            interval_y = points[points[:,1] < current_y + box_y]
            interval_y = interval_y[interval_y[:,1] >= current_y]
            for idx_z in range(grid_z):
                interval_z = interval_y[interval_y[:,2] < current_z + box_z]
                interval_z = interval_z[interval_z[:,2] >= current_z]
                for idx_x in range(grid_x):
                    interval_x = interval_z[interval_z[:,0] < current_x + box_x]
                    interval_x = interval_x[interval_x[:,0] >= current_x]
                    grid_idx = (idx_y,idx_z,idx_x)
                    box_points = []
                    for point in interval_x:
                        box_points.append(Point(point))
                    current_box = Box(box_points, grid_idx)
                    boxes.append(current_box)
                    current_x += box_x
                    f.value += 1
                current_x = min_x
                current_z += box_z
            current_z = min_z
            current_y += box_y
        self.boxes = boxes
        self.get_nonempty_boxes()
        self.get_adjacent_boxes()
        print('\n|Grid Complete|')

In [74]:
class Point:
    
    def __init__(self, position):
        self.x, self.y, self.z = position
        self.idx = None
        self.state = 0
        
    def __eq__(self, other):
        if not isinstance(other, Point): return NotImplemented
        return self.x == other.x and self.y == other.y and self.z == other.z
    
    def __sub__(self, other):
        if not isinstance(other, Point): return NotImplemented
        return (self.x - other.x, self.y - other.y, self.z - other.z)
    
    def __add__(self, other):
        if not isinstance(other, Point): return NotImplemented
        return (self.x + other.x, self.y + other.y, self.z + other.z)
    
    def position(self):
        return (self.x, self.y, self.z)
    
    
class Edge:
    
    def __init__(self, vertices):
        self.vertices = vertices
        self.state = 0
    
    def __eq__(self, other):
        if not isinstance(other, Edge): return NotImplemented
        for vertex in other.vertices:
            if not vertex in self.vertices:
                return False
        return True

    
class Triangle:
    
    def __init__(self, vertices, edges):
        self.vertices = vertices
        self.edges = edges
    
    def angle_fitness(self, vertices):
        v1, v2, v3 = vertices
        for vertex in self.vertices:
            if not vertex in vertices:
                v0 = vertex
        u = Vector(v0 - v1)
        v = Vector(v2 - v1)
        n1 = v.cross(u)
        u = Vector(v3 - v1)
        n2 = u.cross(v)
        return abs(np.tan(n1.angle(n2)))
    
    def expanding_vector(self, edge):
        v1, v2 = edge.vertices
        for vertex in self.vertices:
            if not vertex in edge.vertices:
                v3 = vertex
        w = Vector(v3 - v1)
        v = Vector(v2 - v1)
        v = v.unit_vector()
        e = v * v.dot(w) - w
        return e.unit_vector()
        

class Vector:
    
    def __init__(self, vector):
        self.x, self.y, self.z = vector
    
    def __sub__(self, other):
        return Vector((self.x - other.x, self.y - other.y, self.z - other.z))
    
    def __truediv__(self, constant):
        return Vector((self.x / constant, self.y / constant, self.z / constant))
    
    def __mul__(self, constant):
        return Vector((self.x * constant, self.y * constant, self.z * constant))
    
    def dot(self, other):
        return self.x*other.x + self.y*other.y + self.z*other.z
    
    def cross(self, other):
        x = self.y*other.z-self.z*other.y
        y = self.z*other.x-self.x*other.z
        z = self.x*other.y-self.y*other.x
        return Vector((x, y, z))
    
    def length(self):
        return self.dot(self)**0.5
    
    def angle(self, other):
        cosine_angle = self.dot(other) / (self.length()*other.length())
        return np.arccos(cosine_angle)
    
    def unit_vector(self):
        return self / self.length()
    
    def position(self):
        return (self.x, self.y, self.z)

class Geometries:
    
    def __init__(self):
        self.pcd = None
        self.line_set = None
        self.mesh = None
    
    def add_mesh(self, current_triangles):
        vertices = []
        edges = []
        triangles = []
        for triangle in current_triangles:
            for _, current_vertex in enumerate(triangle.vertices):
                added = False
                for other_idx, other_vertex in enumerate(vertices):
                    if current_vertex is other_vertex:
                        current_vertex.idx = other_idx
                        added = True
                if not added:
                    current_vertex.idx = len(vertices)
                    vertices.append(current_vertex.position())
            v1, v2, v3 = triangle.vertices
            triangles.append([v1.idx, v2.idx, v3.idx])
            for edge in triangle.edges:
                v1, v2 = edge.vertices
                edges.append([v1.idx, v2.idx])
                
        mesh = geometry.TriangleMesh()
        mesh.vertices = utility.Vector3dVector(np.asarray(vertices))
        mesh.triangles = utility.Vector3iVector(np.asarray(triangles))
        self.mesh = mesh
        
        line_set = geometry.LineSet()
        colors = [[1, 0, 0] for i in range(len(edges))]
        line_set.points = utility.Vector3dVector(np.asarray(vertices))
        line_set.lines = utility.Vector2iVector(np.asarray(edges))
        line_set.colors = utility.Vector3dVector(colors)
        self.line_set = line_set
        
    
    def add_pcd(self, points):
        pcd = geometry.PointCloud()
        pcd.points = utility.Vector3dVector(points)
        self.pcd = pcd
    

class Mesh:
    
    def __init__(self):
        self.vertices = []
        self.edges = []
        self.triangles = []
    
    def add_triangle(self, triangle):
        self.triangles.append(triangle)
        for edge in triangle.edges:
            if edge in self.edges:
                edge.state = 1
            else:
                self.edges.append(edge)
        for vertex in triangle.vertices:
            vertex.state = 1
    
    def create_triangle(self, vertices):
        v1, v2, v3 = vertices
        edge1 = Edge((v1, v2))
        edge2 = Edge((v1, v3))
        edge3 = Edge((v2, v3))
        for edge in self.edges:
            if edge == edge1:
                edge1 = edge
            elif edge == edge2:
                edge2 = edge
            elif edge == edge3:
                edge3 = edge
        return Triangle(vertices, (edge1, edge2, edge3))
    
    def find_neighbor(self, vertex):
        open_vertices = [vertex for vertex in self.vertices if vertex.state is 0]
        positions = np.array([vertex.position() for vertex in open_vertices])
        point_cloud = geometry.PointCloud()
        point_cloud.points = utility.Vector3dVector(positions)
        tree = geometry.KDTreeFlann(point_cloud)
        [_, neighbor_idx, _] = tree.search_knn_vector_3d(vertex.position(), 2)
        neighbor = open_vertices[neighbor_idx[1]]
        return neighbor
    
    def find_edge_neighbors(self, v1, v2):
        midpoint = Vector(v1 + v2) / 2
        neighbors = [[vertex, Vector(vertex - Point(midpoint.position())).length()] for vertex in self.vertices if not vertex == v1 and not vertex == v2]
        ordered_neighbors = sorted(neighbors, key=lambda i: i[1])
        return ordered_neighbors
    
    def create_seed_triangle(self):
        v1 = random.choice(self.vertices)
        v2 = self.find_neighbor(v1)
        ordered_neighbors = self.find_edge_neighbors(v1, v2)
        for vertex, _ in ordered_neighbors:
            triangle = self.create_triangle((v1, v2, vertex))
            self.add_triangle(triangle)
            return triangle
    
    def check_triangle(self, triangle):
        return (self.check_angles(triangle) and 
                self.check_lengths(triangle) and 
                self.check_valid_edges(triangle) and 
                self.check_intersection(triangle) and 
                self.check_overlap(triangle))
    
    def check_overlap(self, current_triangle):
        v0, v1, v2 = current_triangle.vertices
        for triangle in self.triangles:
            if v2 in triangle.vertices:
                vector_CA = Vector(v0 - v2)
                vector_CB = Vector(v1 - v2)
                vector_BC = vector_CB*-1
                vector_AC = vector_CA*-1
                n = vector_CB.cross(vector_CA).unit_vector()
                region_BCCA = vector_BC.dot(vector_CA) + 1
                region_CACB = vector_CA.dot(vector_CB) + 1
                region_CBAC = vector_CB.dot(vector_AC) + 1
                adjacent_vectors = []
                for vertex in triangle.vertices:
                    if not vertex == v2:
                        adjacent_vectors.append(Vector(vertex - v2))
                vector_CP1 = adjacent_vectors[0]
                vector_CP2 = adjacent_vectors[1]
                projected_CP1 = n.cross(vector_CP1.cross(n)).unit_vector()
                projected_CP2 = n.cross(vector_CP2.cross(n)).unit_vector()
                if (projected_CP1.dot(vector_CA)+projected_CP1.dot(vector_CB) > region_CACB or 
                    projected_CP2.dot(vector_CA)+projected_CP2.dot(vector_CB) > region_CACB):
                    return False
                if (projected_CP1.dot(vector_BC)+projected_CP1.dot(vector_CA) >= region_BCCA and 
                    projected_CP2.dot(vector_CB)+projected_CP2.dot(vector_AC) >= region_CBAC):
                    if (projected_CP1.dot(vector_CA) - projected_CP1.dot(vector_BC) + 
                        projected_CP2.dot(vector_CB) - projected_CP2.dot(vector_AC) > 0):
                        return False
                elif (projected_CP2.dot(vector_BC)+projected_CP2.dot(vector_CA) >= region_BCCA and 
                      projected_CP1.dot(vector_CB)+projected_CP1.dot(vector_AC) >= region_CBAC):
                    if (projected_CP2.dot(vector_CA) - projected_CP2.dot(vector_BC) + 
                        projected_CP1.dot(vector_CB) - projected_CP1.dot(vector_AC) > 0):
                        return False
        return True
                    
    def check_intersection(self, current_triangle):
        #triangles = self.triangles
        #idx = len(triangles)
        #triangles.append(current_triangle)
        #geometries = Geometries()
        #geometries.add_mesh(triangles)
        #for intersection in np.asarray(geometries.mesh.get_self_intersecting_triangles()):
            #if idx in intersection:
                #return False
        #return True
        for edge in current_triangle.edges:
            p0, p1 = edge.vertices
            for triangle in self.triangles:
                v0, v1, v2 = triangle.vertices
                v0v1 = Vector(v1 - v0)
                v0v2 = Vector(v2 - v0)
                direction = Vector(p1 - p0).unit_vector()
                pvec = direction.cross(v0v2)
                det = v0v1.dot(pvec)
                if det < 1e-8:
                    break
                invDet = 1.0 / det
                tvec = Vector(p0 - v0)
                u = tvec.dot(pvec) * invDet
                if u < 0 or u > 1:
                    break
                qvec = tvec.cross(v0v1)
                v = direction.dot(qvec) * invDet
                if v < 0 or u + v > 1:
                    break
                return False
        for triangle in self.triangles:
            for edge in triangle.edges:
                p0, p1 = edge.vertices
                v0, v1, v2 = current_triangle.vertices
                v0v1 = Vector(v1 - v0)
                v0v2 = Vector(v2 - v0)
                direction = Vector(p1 - p0).unit_vector()
                pvec = direction.cross(v0v2)
                det = v0v1.dot(pvec)
                if det < 1e-8:
                    break
                invDet = 1.0 / det
                tvec = Vector(p0 - v0)
                u = tvec.dot(pvec) * invDet
                if u < 0 or u > 1:
                    break
                qvec = tvec.cross(v0v1)
                v = direction.dot(qvec) * invDet
                if v < 0 or u + v > 1:
                    break
                return False
        return True
    
    def check_angles(self, triangle):
        v1, v2, v3 = triangle.vertices
        angles = []
        u = Vector(v2 - v1)
        w = Vector(v3 - v1)
        angles.append(np.degrees(u.angle(w)))
        u = Vector(v1 - v2)
        w = Vector(v3 - v2)
        angles.append(np.degrees(u.angle(w)))
        u = Vector(v1 - v3)
        w = Vector(v2 - v3)
        angles.append(np.degrees(u.angle(w)))
        for angle in angles:
            if angle < self.min_triangle_angle or angle > self.max_triangle_angle:
                return False
        return True
    
    def check_lengths(self, triangle):
        v1, v2, v3 = triangle.vertices
        vectors = []
        vectors.append(Vector(v2 - v1))
        vectors.append(Vector(v3 - v2))
        vectors.append(Vector(v1 - v3))
        for vector in vectors:
            if vector.length() < self.min_edge_length or vector.length() > self.max_edge_length:
                return False
        return True
    
    def check_valid_edges(self, current_triangle):
        #triangles = self.triangles
        #idx = len(triangles)
        #triangles.append(current_triangle)
        #geometries = Geometries()
        #geometries.add_mesh(triangles)
        #for edge in np.asarray(geometries.mesh.current_triangle)
        for edge in current_triangle.edges:
            count = 0
            for other_edge in self.edges:
                if other_edge == edge:
                    count += 1
            if count > 1:
                return False
            #if edge.state is 1:
                #return False
        return True
    
    def triangulate_edges(self, parent_triangle):
        child_triangles = []
        for edge in parent_triangle.edges:
            if edge.state is 0:
                edge_vector = parent_triangle.expanding_vector(edge)
                v1, v2 = edge.vertices
                midpoint = Vector(v1 + v2) / 2
                potential_vertices = []
                for vertex in self.vertices:
                    if not vertex in edge.vertices:
                        if edge_vector.dot(Vector(vertex - v1)) > 0:
                            d = Vector(vertex.position()) - midpoint
                            t = parent_triangle.angle_fitness((v1, v2, vertex))
                            fitness = t*d.length()
                            potential_vertices.append([vertex, fitness])
                ordered_vertices = sorted(potential_vertices, key=lambda i: i[1])
                triangle_created = False
                for vertex, _ in ordered_vertices:
                    if not triangle_created:
                        triangle = self.create_triangle((v1, v2, vertex))
                        if self.check_triangle(triangle):
                            self.add_triangle(triangle)
                            child_triangles.append(triangle)
                            triangle_created = True
        return child_triangles
            
    def triangulate(self, parameters):
        self.min_edge_length, self.max_edge_length, self.min_triangle_angle, self.max_triangle_angle = parameters
        current_triangles = [self.create_seed_triangle()]
        while len(current_triangles) > 0:
            next_triangles = []
            for triangle in current_triangles:
                next_triangles.extend(self.triangulate_edges(triangle))
            current_triangles = next_triangles
    

class PointCloud:
    
    def __init__(self):
        self.points = []
        self.grid = None
        self.pcd = None
        
    def read(self, filedir):
        try:
            self.points = np.asarray(io.read_point_cloud(filedir).points)
        except OSError:
            print('File Directory does not exist')
        
    def rotate(self, angle):
        points = self.points
        x = points[:,0]
        y = points[:,2] + abs(points[:,2].min())
        z = points[:,1]
        points = np.array([x, y, z]).transpose()
        angle = math.radians(angle)
        theta = [0,angle,0]
        r_x = np.array([[1,0,0],[0,math.cos(theta[0]),-math.sin(theta[0])],[0,math.sin(theta[0]),math.cos(theta[0])]])
        r_y = np.array([[math.cos(theta[1]),0,math.sin(theta[1])],[0,1,0],[-math.sin(theta[1]),0,math.cos(theta[1])]])
        r_z = np.array([[math.cos(theta[2]),-math.sin(theta[2]),0],[math.sin(theta[2]),math.cos(theta[2]),0],[0,0,1]])   
        r = np.dot(r_z, np.dot(r_y,r_x))
        self.points = np.matmul(points, r)
        pcd = geometry.PointCloud()
        pcd.points = utility.Vector3dVector(self.points)
        self.pcd = pcd
        
    def create_grid(self, density=20):
        self.grid = Grid(self.points, density)
        
    def triangulate(self, density=20, min_edge_length=1e-6, max_edge_length=1e6, min_triangle_angle=1e-6, max_triangle_angle=180):
        parameters = (min_edge_length, max_edge_length, min_triangle_angle, max_triangle_angle)
        self.create_grid(density=density)
        print('|Triangulating Point Cloud|')
        f = FloatProgress(min=0, max=len(self.grid.nonempty_boxes))
        display(f)
        #
        ch_vertices = []
        ch_triangles = []
        for current_box in self.grid.nonempty_boxes:
            mesh = current_box.convex_hull()
            if not mesh is None:
                current_vertices = np.asarray(mesh.vertices)
                current_triangles = np.asarray(mesh.triangles)
                for triangle in current_triangles:
                    new_triangle = []
                    for vertex_idx in triangle:
                        new_triangle.append(len(ch_vertices))
                        ch_vertices.append(current_vertices[vertex_idx])
                    ch_triangles.append(new_triangle)
        mesh = geometry.TriangleMesh()
        mesh.vertices = utility.Vector3dVector(np.asarray(ch_vertices))
        mesh.triangles = utility.Vector3iVector(np.asarray(ch_triangles))
        #
        #triangles = []
        #for current_box in self.grid.nonempty_boxes:
        #    current_box.triangulate(parameters)
        #    triangles.extend(current_box.mesh.triangles)
        #    f.value += 1
        #print('|Triangulation Complete|')
        #geometries = Geometries()
        #geometries.add_mesh(triangles)
        #geometries.add_pcd(self.points)
        return mesh#, geometries

In [ ]:
folderdir = '/Users/nickf/Desktop/Custom-Fit/Data/Body-Datasets/Caesar-Data/CSRFM_PCD/'
filedirs = glob.glob(f'{folderdir}/*.ply')
file_num = 10
point_cloud = PointCloud()
point_cloud.read(filedirs[file_num])
point_cloud.rotate(232)

In [68]:
# density of 13 yielded less conflicting submeshes but fewer triangles
mesh = point_cloud.triangulate(density=13)

|Starting Grid|

Number of Neighbors: 13
Number of Boxes: 6992
Grid Dimensions: [23 38  8]
Box Dimensions: [36.87054976 47.26808178 35.8402313 ]


FloatProgress(value=0.0, max=6992.0)


|Grid Complete|
|Triangulating Point Cloud|


FloatProgress(value=0.0, max=931.0)

In [69]:
render = visualization.RenderOption()
filename = '/Users/nickf/Desktop/Custom-Fit/Scripts/render.json'
render.load_from_json(filename)

In [57]:
visualization.draw_geometries([geometries.mesh, geometries.line_set, geometries.pcd])

In [77]:
line_set = geometry.create_line_set_from_triangle_mesh(mesh)
visualization.draw_geometries([point_cloud.pcd, mesh, line_set])

In [ ]:
#vis.create_window()
#vis.add_geometry(geometries.mesh)
#vis.add_geometry(geometries.line_set)
#vis.run()

In [ ]:
# Check Overlap needs to be changed - it doesn't work at all